# `std::tuple`

This is a generalization of pairs to any number of elements.

In [16]:
#include <iostream>
#include <string>
#include <tuple>

In [17]:
{
    std::tuple<int,double,std::string> t1 { 5, 6.8, "chose" } ;
    // .....
    int i = std::get<0>(t1) ;  // i is equal to 5
    std::get<1>(t1) = 2.3 ;    // remplaces the 2nd element 6.8 by 2.3
    auto s = std::get<2>(t1) ; // s will be a string and will contain "chose"
    std::cout<<i<<" "<<std::get<1>(t1)<<" "<<s<<std::endl ;
}

5 2.3 chose


A tuple can obviously be assigned to another tuple of the same type, but also in case of different types, if there is a legal implicit conversion for each of the elements.

WARNING: the `tuple` is not a list, and there is no way to iterate over its values at runtime. Let's put it another way: since the type of the value can change from one iteration to another, it is impossible to compile once and for all the instructions that would be in such a loop. Of course, with template-based meta-programming, the developer can rely on the compiler to unroll the loop at the compilation, but that's not really in the spirit of `std::tuple`.

In fact, the `std::tuple` is now the ideal type to return a collection of results, and nothing more prevents the developper from doing it: the RVO (return value optimization) avoid any unnecessary copy of the returned data, and type inference makes it easier for the developer:

In [18]:
#include <stdexcept> 

In [19]:
std::tuple<double, char, std::string> get_student( int id )
 {
  if (id == 0) return { 3.8, 'A', "Lisa Simpson" };
  if (id == 1) return { 2.9, 'C', "Milhouse Van Houten" };
  if (id == 2) return { 1.7, 'D', "Ralph Wiggum" };
  throw std::invalid_argument("unknown student id");
 }

In [20]:
void display_student( int id )
 {
  auto student = get_student(id);
  std::cout
    << "GPA: " << std::get<0>(student) << ", "
    << "grade: " << std::get<1>(student) << ", "
    << "name: " << std::get<2>(student)
    << std::endl;
 }

In [21]:
{
  display_student(0) ;
  display_student(1) ;
  display_student(2) ;
}

GPA: 3.8, grade: A, name: Lisa Simpson
GPA: 2.9, grade: C, name: Milhouse Van Houten
GPA: 1.7, grade: D, name: Ralph Wiggum


## `std::tie`

When the values are only read, the `std::tie` function allows to attach the member variables of a tuple to independent variables, which allows, in a way, to give names to the member variables and to have a slightly more readable code. But only for reading.

In [22]:
void display_student( int id )
 {
  double gpa ;
  char grade ;
  std::string name ;
  std::tie(gpa,grade,name) = get_student(id) ;

  std::cout
    << "GPA: " << gpa << ", "
    << "grade: " << grade << ", "
    << "name: " << name
    << std::endl;
 }

In [23]:
{
  display_student(0) ;
  display_student(1) ;
  display_student(2) ;
}

GPA: 3.8, grade: A, name: Lisa Simpson
GPA: 2.9, grade: C, name: Milhouse Van Houten
GPA: 1.7, grade: D, name: Ralph Wiggum


## C++17 structured binding

Since C++17, std::tie can be put aside and one can take advantage of on-the-fly "unpacking" :

In [24]:
void display_student( int id )
 {
  auto [ gpa, grade, name ] = get_student(id) ;
    
  std::cout
    << "GPA: " << gpa << ", "
    << "grade: " << grade << ", "
    << "name: " << name
    << std::endl;
 }

In [25]:
{
  display_student(0) ;
  display_student(1) ;
  display_student(2) ;
}

GPA: 3.8, grade: A, name: Lisa Simpson
GPA: 2.9, grade: C, name: Milhouse Van Houten
GPA: 1.7, grade: D, name: Ralph Wiggum


## Better than a good old-fashioned `struct` ?

In most cases, would it be easier and more readable to create a simple `struct` ?

In [26]:
struct Student
{
  double gpa ;
  char grade ;
  std::string name ;
} ;

In [27]:
Student get_student(int id)
{
  if (id == 0) return { 3.8, 'A', "Lisa Simpson" } ;
  if (id == 1) return { 2.9, 'C', "Milhouse Van Houten" } ;
  if (id == 2) return { 1.7, 'D', "Ralph Wiggum" } ;
  throw std::invalid_argument("id");
}

In [28]:
void display_student( int id )
 {
  auto s = get_student(id) ;
    
  std::cout
    << "GPA: " << s.gpa << ", "
    << "grade: " << s.grade << ", "
    << "name: " << s.name
    << std::endl;
 }

In [29]:
{
  display_student(0) ;
  display_student(1) ;
  display_student(2) ;
}

GPA: 3.8, grade: A, name: Lisa Simpson
GPA: 2.9, grade: C, name: Milhouse Van Houten
GPA: 1.7, grade: D, name: Ralph Wiggum


In reality, the `std::tuple` only avoid the definition of the structure. As a consequence, the developer must use "structure bindgins " to keep their code clear. The `std::tuple` only becomes really interesting if the developer needs to write a function which must iterate over all the elements of a tuple, without knowing in advance the number and type of each element, which can never be done on an ordinary structure.

However, since each element of the tuple is of potentially different type, it is impossible to write a simple `for` loop that would be unrolled at the runtime. It is necessary to unroll the loop at compilation, with template mechanisms. Below is an example that displays all the elements of a tuple. If ever a new element is added to the tuple, it is no longer necessary to modify the main program :

In [1]:
#include <iostream>
#include <tuple>

In [2]:
std::tuple<double, char, std::string> get_student( int id )
 {
  if (id == 0) return { 3.8, 'A', "Lisa Simpson" };
  if (id == 1) return { 2.9, 'C', "Milhouse Van Houten" };
  if (id == 2) return { 1.7, 'D', "Ralph Wiggum" };
  throw std::invalid_argument("unknown student id");
 }

In [3]:
void display_student( int id )
 {
  std::apply
   (
    []( auto const &... elem )
    { ((std::cout<<elem<<" "), ...) ; },
    get_student(id)
   ) ;
  std::cout<<std::endl ;
 }

In [4]:
{
  display_student(0) ;
  display_student(1) ;
  display_student(2) ;
}

3.8 A Lisa Simpson 
2.9 C Milhouse Van Houten 
1.7 D Ralph Wiggum 


© *CNRS 2020*  
*Assembled and written in french by David Chamont, translated by Karim Asnaoui, this work is made available according to the terms of the*  
[*Creative Commons License - Attribution - NonCommercial - ShareAlike 4.0 International*](http://creativecommons.org/licenses/by-nc-sa/4.0/)